In [ ]:
#!eval-notebook "InitSystemorphBaseToMemory"

In [ ]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());

In [ ]:
Workspace.InitializeFrom(DataSource);

In [ ]:
await UploadOpeningAsync("../Files/900.TransactionalData/Openings_CH_2020_12.csv")

In [ ]:
var fileName = "../Files/900.TransactionalData/NominalCashflows_CH_2020_12.csv";
Activity.Start();
var args = (await GetArgsFromMainAsync<PartitionByReportingNodeAndPeriod>(fileName)) with {ImportFormat = ImportFormats.Cashflow};
await DataNodeFactoryAsync(fileName, ImportFormats.Cashflow, args);
if(Activity.HasErrors()) return Activity.Finish();
    
var parsingLog = await ParseCashflowsToWorkspaceAsync(fileName, args);

var targetPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                   p.Year == args.Year &&
                                                                                                   p.Month == args.Month &&
                                                                                                   p.Scenario == args.Scenario).ToArrayAsync()).Single().Id;
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(targetPartition);                                                                                            
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(targetPartition);
var parsedVariables = await Workspace.Query<RawVariable>().ToArrayAsync();
var savedVariables = await DataSource.Query<RawVariable>().ToArrayAsync();
var updatedVariables = savedVariables?.Union(parsedVariables, DataRecordComparer<RawVariable>.Instance) ?? Enumerable.Empty<RawVariable>();
var newVariables = updatedVariables?.Except(parsedVariables, DataRecordComparer<RawVariable>.Instance) ?? Enumerable.Empty<RawVariable>();
if(newVariables.Count() == 0) {
    ApplicationMessage.Log(Warning.VariablesAlreadyImported, "");
    return Activity.Finish().Merge(parsingLog);
    }
return Activity.Finish().Merge(parsingLog);